# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

nltk.download(['wordnet', 'punkt', 'stopwords'])



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\missn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\missn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\missn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_messages_database.db')
df = pd.read_sql_table('disaster_messages_table', engine)
X = df['message'] # Message Column
Y = df.iloc[:, 4:] # Classification label

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
     """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the message
    Return:
      lemm(list of str): a list of the root form of the message words
    """
        
    # Normalize text
     text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
     words = word_tokenize(text)
    
    # Remove stop words
     stop = stopwords.words("english")
    words = [t for t in words if t not in stop]
    
    # Lemmatization
     lemm = [WordNetLemmatizer().lemmatize(w) for w in words]
    
     return lemm

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Create a pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Spliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Fit the pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Predicting
y_pred = pipeline.predict(X_test)

In [8]:
# Testing the model
# Printing the classification report for each label

i = 0
for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1

Feature 1: related
              precision    recall  f1-score   support

           0       0.71      0.40      0.51      1540
           1       0.83      0.95      0.89      4970
           2       0.55      0.27      0.36        44

    accuracy                           0.82      6554
   macro avg       0.70      0.54      0.59      6554
weighted avg       0.80      0.82      0.79      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5421
           1       0.85      0.50      0.63      1133

    accuracy                           0.90      6554
   macro avg       0.88      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       

C:\Users\missn\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Feature 8: security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6432
           1       0.25      0.01      0.02       122

    accuracy                           0.98      6554
   macro avg       0.62      0.50      0.50      6554
weighted avg       0.97      0.98      0.97      6554

Feature 9: military
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6311
           1       0.69      0.04      0.07       243

    accuracy                           0.96      6554
   macro avg       0.83      0.52      0.53      6554
weighted avg       0.95      0.96      0.95      6554

Feature 10: child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

Feature 


Feature 32: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6483
           1       1.00      0.01      0.03        71

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.98      6554

Feature 33: earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5951
           1       0.88      0.78      0.83       603

    accuracy                           0.97      6554
   macro avg       0.93      0.88      0.90      6554
weighted avg       0.97      0.97      0.97      6554

Feature 34: cold
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6422
           1       0.78      0.05      0.10       132

    accuracy                           0.98      6554
   macro avg       0.88      0.53      0.54      6554
weighted avg    

In [9]:
accuracy = (y_pred == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(accuracy))

The model accuracy is 0.948


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# Show parameters for the pipline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000013679BC8D38>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [11]:
# Create Grid search parameters
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline, param_grid = parameters)
cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# Fit the tuned model
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [13]:
# Predicting
y_pred = cv.predict(X_test)

# Testing the model
# Printing the classification report for each label
print('The results of the tuned model')
i = 0
for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
        
accuracy = (y_pred == y_test.values).mean()
print('The tuned model accuracy is {:.3f}'.format(accuracy))

The results of the tuned model
Feature 1: related
              precision    recall  f1-score   support

           0       0.69      0.42      0.52      1540
           1       0.83      0.94      0.89      4970
           2       0.50      0.20      0.29        44

    accuracy                           0.81      6554
   macro avg       0.68      0.52      0.57      6554
weighted avg       0.80      0.81      0.80      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5421
           1       0.82      0.45      0.58      1133

    accuracy                           0.89      6554
   macro avg       0.86      0.72      0.76      6554
weighted avg       0.88      0.89      0.87      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.0

C:\Users\missn\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Feature 7: search_and_rescue
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6372
           1       0.62      0.08      0.15       182

    accuracy                           0.97      6554
   macro avg       0.80      0.54      0.57      6554
weighted avg       0.96      0.97      0.96      6554

Feature 8: security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6432
           1       0.33      0.02      0.03       122

    accuracy                           0.98      6554
   macro avg       0.66      0.51      0.51      6554
weighted avg       0.97      0.98      0.97      6554

Feature 9: military
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6311
           1       0.93      0.05      0.10       243

    accuracy                           0.96      6554
   macro avg       0.95      0.53      0.54      6554
weig


Feature 36: direct_report
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      5277
           1       0.76      0.34      0.47      1277

    accuracy                           0.85      6554
   macro avg       0.81      0.66      0.69      6554
weighted avg       0.84      0.85      0.83      6554

The tuned model accuracy is 0.947


In [14]:
# Showing the best parameters
print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf__estimator__n_estimators': 20, 'tfidf__use_idf': True}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Creating a new pipline to improve the model
pipeline2 = Pipeline([
        ('tfidfvect', TfidfVectorizer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [16]:
# Fit the improved model
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidfvect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      

In [17]:
# Predicting
y_pred = pipeline2.predict(X_test)

# Testing the model
# Printing the classification report for each label
print('The results of the improved model')
i = 0
for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
        
accuracy = (y_pred == y_test.values).mean()
print('The improved model accuracy is {:.3f}'.format(accuracy))

The results of the improved model
Feature 1: related
              precision    recall  f1-score   support

           0       0.58      0.17      0.26      1540
           1       0.78      0.96      0.86      4970
           2       0.36      0.20      0.26        44

    accuracy                           0.77      6554
   macro avg       0.58      0.44      0.46      6554
weighted avg       0.73      0.77      0.72      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5421
           1       0.78      0.54      0.64      1133

    accuracy                           0.89      6554
   macro avg       0.84      0.75      0.79      6554
weighted avg       0.89      0.89      0.89      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.11      0.03      0.05        29

    accuracy                           


Feature 30: floods
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      6044
           1       0.85      0.59      0.69       510

    accuracy                           0.96      6554
   macro avg       0.91      0.79      0.84      6554
weighted avg       0.96      0.96      0.96      6554

Feature 31: storm
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5977
           1       0.73      0.45      0.56       577

    accuracy                           0.94      6554
   macro avg       0.84      0.72      0.76      6554
weighted avg       0.93      0.94      0.93      6554

Feature 32: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6483
           1       0.54      0.21      0.30        71

    accuracy                           0.99      6554
   macro avg       0.76      0.60      0.65      6554
weighted avg       

In [18]:
# Show parameters for the pipline
pipeline2.get_params()

{'memory': None,
 'steps': [('tfidfvect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.float64'>, encoding='utf-8',
                   input='content', lowercase=True, max_df=1.0, max_features=None,
                   min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                   smooth_idf=True, stop_words=None, strip_accents=None,
                   sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=None, use_idf=True, vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=50,
                                                      random_state=None),
                         n_jobs=None))],
 'verbose': False,
 '

In [19]:
# Using grid search
# Create Grid search parameters
parameters2 = {
        'tfidfvect__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]
}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2)

# Fit the  model
cv2.fit(X_train, y_train)

# Predicting
y_pred = cv2.predict(X_test)

# Testing the model
# Printing the classification report for each label
print('The results of the model')
i = 0
for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
        
accuracy = (y_pred == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(accuracy))

# Showing the best parameters
print("Best Parameters:", cv2.best_params_)


The results of the model
Feature 1: related
              precision    recall  f1-score   support

           0       0.60      0.24      0.34      1540
           1       0.80      0.95      0.87      4970
           2       0.54      0.32      0.40        44

    accuracy                           0.78      6554
   macro avg       0.65      0.50      0.54      6554
weighted avg       0.75      0.78      0.74      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5421
           1       0.77      0.55      0.64      1133

    accuracy                           0.89      6554
   macro avg       0.84      0.76      0.79      6554
weighted avg       0.89      0.89      0.89      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.29      0.07      0.11        29

    accuracy                           1.00     


Feature 26: shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6518
           1       0.17      0.03      0.05        36

    accuracy                           0.99      6554
   macro avg       0.58      0.51      0.52      6554
weighted avg       0.99      0.99      0.99      6554

Feature 27: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6485
           1       0.27      0.12      0.16        69

    accuracy                           0.99      6554
   macro avg       0.63      0.56      0.58      6554
weighted avg       0.98      0.99      0.98      6554

Feature 28: other_infrastructure
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6266
           1       0.33      0.09      0.14       288

    accuracy                           0.95      6554
   macro avg       0.64      0.54      0.56      655

### 9. Export your model as a pickle file

In [20]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f) 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.